# Load data from server

## 1. Import packages

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from astroquery.vizier import Vizier

In [ ]:
import random
import sys
import os


from math import *
import matplotlib as mpl

plt.rc('font', family='serif')
mpl.rcParams.update({'font.size': 12})
mpl.rcParams.update({'legend.labelspacing':0.25, 'legend.fontsize': 12})
mpl.rcParams.update({'errorbar.capsize': 4})

## 2. Load data

In [ ]:
Vizier.ROW_LIMIT = 76 #30000
catalogs = Vizier.get_catalogs('J/A+A/618/A93/members')  # astroquery.utils.commons.TableList
print(catalogs)

Since `catalogs` only has 1 table, we are only interested in `catalogs[0]`. We will transform it to a `pandas` Data Frame for ease of handling.

In [ ]:
catalog = catalogs[0].to_pandas()
catalog = catalog.applymap(lambda x: x.decode() if isinstance(x, bytes) else x)
catalog

## 3. Create examples folder

In [ ]:
if not os.path.exists('examples'):
    os.makedirs('examples')
if not os.path.exists('neg_examples'):
    os.makedirs('neg_examples')

## 4. 

In [ ]:
cluster_name = []
len(cluster_name)

In [ ]:
cluster_name.append(catalogs[0]['Cluster'][0])

In [ ]:
cluster_name[-1]

In [ ]:
gmag

In [ ]:
tr_matrix

In [ ]:


clusters = np.unique(catalogs[0]['Cluster'])
number_clusters = len(clusters)



In [ ]:
number_clusters

In [ ]:
number_clusters = np.shape(catalogs[0])[0]

In [ ]:
str(catalogs[0]['Gmag'][10]).find('--')

In [ ]:
number_clusters = np.shape(catalogs[0])[0]

# initialize
cluster_name = []
tr_matrix = []
#t = 0
for i in np.arange(0, number_clusters):
    if len(cluster_name) == 0: # initialization
        #print(f'start {i}')
        gmag = []
        bp_rp = []
        cluster_name.append(catalogs[0]['Cluster'][i])
        
    print(i, cluster_name[-1], catalogs[0]['Cluster'][i])

    if len(cluster_name) > 0 and cluster_name[-1] == catalogs[0]['Cluster'][i]:
        #print(f'medium {i}')
        if (str(catalogs[0]['Gmag'][i]).find('--') < 0) and (str(catalogs[0]['BP-RP'][i]).find('--') < 0):
            gmag.append (float(catalogs[0]['Gmag'][i]))
            bp_rp.append (float(catalogs[0]['BP-RP'][i]))  

    if len(cluster_name) > 0 and cluster_name[-1] != catalogs[0]['Cluster'][i]:
        print(f'end {i}')
        plt.scatter (bp_rp, gmag)
        plt.xlabel(r'$B_p - R_p$ (mag)')
        plt.ylabel(r'$G$ (mag)')
        plt.ylim([18,8])
        plt.savefig('Alessi_12.pdf')
        plt.show()
        
        gmax = np.max (gmag)
        gmin = np.min (gmag)
        cmax = np.max (bp_rp)
        cmin = np.min (bp_rp)
        #print ('Values: ', gmax, gmin, cmax, cmin)

        H, xedges, yedges = np.histogram2d (bp_rp, gmag, bins=20, density=True)

        plt.imshow(H, interpolation='nearest')
        plt.savefig ('examples/'+str(cluster_name[-1])+'.jpg')
        plt.cla()
        plt.clf()
        plt.close()
        plt.show()

        H1 = H.reshape ((400))

        for j in range (0, len(H1)):
            print ('--> ', j, H1[j])

        tr_matrix.append(H1)
        
        #break

        gmag = []
        bp_rp = []
        cluster_name.append(catalogs[0]['Cluster'][i])

    #print (i, catalogs[0]['Cluster'][i])


#np.save ('matrix_real', tr_matrix)

In [ ]:
test_file = 'matrix_real.npy'
test_data = np.load(test_file, allow_pickle=True)
test_data

In [ ]:
cluster_name = []
tr_matrix = []

#create dir for examples
if not os.path.exists('examples'):
    os.makedirs('examples')
if not os.path.exists('neg_examples'):
    os.makedirs('neg_examples')


t = 0
for i in range (0, len(catalogs[0]['Cluster'])):

	if len(cluster_name) > 0 and cluster_name[-1] == catalogs[0]['Cluster'][i]:

		#print (str(catalogs[0]['Gmag'][i]), str(catalogs[0]['BP-RP'][i]))

		if (str(catalogs[0]['Gmag'][i]).find('--') < 0) and (str(catalogs[0]['BP-RP'][i]).find('--') < 0):
			#print ('NAN')
			#sys.exit(0)

			gm.append (float(catalogs[0]['Gmag'][i]))		
			cl.append (float(catalogs[0]['BP-RP'][i]))  
		


	if len(cluster_name) > 0 and cluster_name[-1] != catalogs[0]['Cluster'][i]:

		plt.scatter (cl, gm)
		plt.xlabel(r'$B_p - R_p$ (mag)')
		plt.ylabel(r'$G$ (mag)')
		plt.ylim([18,8])
		plt.savefig('Alessi_12.pdf')
		plt.show()

		gmax = np.max (gm)
		gmin = np.min (gm)
		cmax = np.max (cl)
		cmin = np.min (cl)


		print ('Values: ', gmax, gmin, cmax, cmin)

		H, xedges, yedges = np.histogram2d (cl, gm, bins=20, density=True)

		plt.imshow(H, interpolation='nearest')
		plt.savefig ('examples/'+str(cluster_name[-1])+'.jpg')
		plt.cla()
		plt.clf()
		plt.close()
		#plt.show()

		H1 = H.reshape ((400))

		for j in range (0, len(H1)):

			print ('--> ', j, H1[j])

		tr_matrix.append (H1)
	
		#break

		gm = []
		cl = []
		cluster_name.append(catalogs[0]['Cluster'][i])


	if len(cluster_name) == 0:
		gm = []
		cl = []
		cluster_name.append(catalogs[0]['Cluster'][i])


	print (i, catalogs[0]['Cluster'][i])


In [ ]:
cluster_name = []
tr_matrix = []

t = 0
for i in range (0, len(catalogs[0]['Cluster'])):

	if len(cluster_name) > 0 and cluster_name[-1] == catalogs[0]['Cluster'][i]:

		#print (str(catalogs[0]['Gmag'][i]), str(catalogs[0]['BP-RP'][i]))

		if (str(catalogs[0]['Gmag'][i]).find('--') < 0) and (str(catalogs[0]['BP-RP'][i]).find('--') < 0):
			#print ('NAN')
			#sys.exit(0)

			gm.append (float(catalogs[0]['Gmag'][i]))		
			cl.append (float(catalogs[0]['BP-RP'][i]))  
		


	if len(cluster_name) > 0 and cluster_name[-1] != catalogs[0]['Cluster'][i]:

		#plt.scatter (cl, gm)
		#plt.xlabel(r'$B_p - R_p$ (mag)')
		#plt.ylabel(r'$G$ (mag)')
		#plt.ylim([18,8])
		#plt.savefig('Alessi_12.pdf')
		#plt.show()

		#gmax = np.max (gm)
		#gmin = np.min (gm)
		#cmax = np.max (cl)
		#cmin = np.min (cl)


		#print ('Values: ', gmax, gmin, cmax, cmin)

		H, xedges, yedges = np.histogram2d (cl, gm, bins=20, density=True)

		plt.imshow(H, interpolation='nearest')
		#plt.savefig ('examples/'+str(cluster_name[-1])+'.jpg')
		plt.cla()
		plt.clf()
		plt.close()
		#plt.show()

		H1 = H.reshape ((400))

		for j in range (0, len(H1)):

			print ('--> ', j, H1[j])

		tr_matrix.append (H1)
	
		#break

		gm = []
		cl = []
		cluster_name.append(catalogs[0]['Cluster'][i])


	if len(cluster_name) == 0:
		gm = []
		cl = []
		cluster_name.append(catalogs[0]['Cluster'][i])


	print (i, catalogs[0]['Cluster'][i])


#np.save ('matrix_real', tr_matrix)

false_mtr = []

print('Number of positive samples: ', len(tr_matrix))

for i in range (0,len(tr_matrix)):

	n = int(random.uniform (30, 500)) ## number of stars

	gm = np.random.uniform (8, 18, size = n)	
	cl = np.random.uniform (0, 5, size = n)

	H, xedges, yedges = np.histogram2d (cl, gm, bins=20, density=True)

	if i <= 10:
		plt.imshow(H, interpolation='nearest')
		#plt.savefig (f'neg_examples/{i}.jpg')
		plt.cla()
		plt.clf()
		plt.close()

	H1 = H.reshape ((400))

	false_mtr.append (H1)